In [1]:
spark

Waiting for a Spark session to start...

Spark application_1533622723243_14892: Some(http://gw02.itversity.com:4052)

In [2]:
import scala.collection.Map
import scala.collection.mutable.ArrayBuffer
import scala.util.Random
import org.apache.spark.broadcast.Broadcast
import org.apache.spark.ml.recommendation.{ALS, ALSModel}
import org.apache.spark.sql.{DataFrame, Dataset, SparkSession}
import org.apache.spark.sql.functions._

In [3]:
spark.sparkContext.getConf.getAll.foreach(x=>if(x._2.contains("/proxy/")){println(x._2)})

Waiting for a Spark session to start...

http://rm01.itversity.com:19288/proxy/application_1533622723243_14892


In [4]:
spark.sparkContext.setCheckpointDir("hdfs:///tmp/")

In [5]:
val base = "hdfs:///user/kranthidr/dataSets/recommendations/audio_profiledata_06-May-2005/"
val rawUserArtistData = spark.read.textFile(base + "user_artist_data.txt")
val rawArtistData = spark.read.textFile(base + "artist_data.txt")
val rawArtistAlias = spark.read.textFile(base + "artist_alias.txt")

base = hdfs:///user/kranthidr/dataSets/recommendations/audio_profiledata_06-May-2005/
rawUserArtistData = [value: string]
rawArtistData = [value: string]
rawArtistAlias = [value: string]


[value: string]

In [6]:
rawUserArtistData.count()

24296858

In [7]:
rawArtistData.count()

1848707

In [8]:
rawArtistAlias.count()

193027

In [9]:
rawUserArtistData.take(5).foreach(println)

1000002 1 55
1000002 1000006 33
1000002 1000007 8
1000002 1000009 144
1000002 1000010 314


In [10]:
rawArtistData.take(5).foreach(println)

1134999	06Crazy Life
6821360	Pang Nakarin
10113088	Terfel, Bartoli- Mozart: Don
10151459	The Flaming Sidebur
6826647	Bodenstandig 3000


In [11]:
rawArtistAlias.take(5).foreach(println)

1092764	1000311
1095122	1000557
6708070	1007267
10088054	1042317
1195917	1042317


In [12]:
def buildCounts(
      rawUserArtistData: Dataset[String],
      bArtistAlias: Broadcast[Map[Int,Int]]): DataFrame = {
    rawUserArtistData.map { line =>
      val Array(userID, artistID, count) = line.split(' ').map(_.toInt)
      val finalArtistID = bArtistAlias.value.getOrElse(artistID, artistID)
      (userID, finalArtistID, count)
    }.toDF("user", "artist", "count")
  }

buildCounts: (rawUserArtistData: org.apache.spark.sql.Dataset[String], bArtistAlias: org.apache.spark.broadcast.Broadcast[scala.collection.Map[Int,Int]])org.apache.spark.sql.DataFrame


In [13]:
def buildArtistAlias(rawArtistAlias: Dataset[String]): Map[Int,Int] = {
    rawArtistAlias.flatMap { line =>
      val Array(artist, alias) = line.split('\t')
      if (artist.isEmpty) {
        None
      } else {
        Some((artist.toInt, alias.toInt))
      }
    }.collect().toMap
  }

buildArtistAlias: (rawArtistAlias: org.apache.spark.sql.Dataset[String])scala.collection.Map[Int,Int]


In [14]:
val bArtistAlias = spark.sparkContext.broadcast(buildArtistAlias(rawArtistAlias))

bArtistAlias = Broadcast(17)


Broadcast(17)

In [15]:
val allData = buildCounts(rawUserArtistData, bArtistAlias)
val Array(trainData, cvData) = allData.randomSplit(Array(0.9, 0.1))
trainData.cache()
cvData.cache()

allData = [user: int, artist: int ... 1 more field]
trainData = [user: int, artist: int ... 1 more field]
cvData = [user: int, artist: int ... 1 more field]


[user: int, artist: int ... 1 more field]

In [16]:
val bAllArtistIDs = spark.sparkContext.broadcast(allData.select("artist").as[Int].distinct().collect())

bAllArtistIDs = Broadcast(23)


Broadcast(23)

In [17]:
import org.apache.spark.ml.evaluation.RegressionEvaluator

In [18]:
// val evaluations =
//       for (rank     <- Seq(5,  30);
//            regParam <- Seq(1.0, 0.0001);
//            alpha    <- Seq(1.0, 40.0))
//       yield {
//         val model = new ALS().
//           setSeed(Random.nextLong()).
//           setImplicitPrefs(true).
//           setRank(rank).setRegParam(regParam).
//           setAlpha(alpha).setMaxIter(20).
//           setUserCol("user").setItemCol("artist").
//           setRatingCol("count").
//           setColdStartStrategy("drop").
//           fit(trainData)
//         val predictions = model.transform(cvData)
//         val evaluator = new RegressionEvaluator().setMetricName("rmse").setLabelCol("count").setPredictionCol("prediction")
//         val rmse = evaluator.evaluate(predictions)
          
//         model.userFactors.unpersist()
//         model.itemFactors.unpersist()
//         println(rmse, rank, regParam, alpha)
//         (rmse, (rank, regParam, alpha))
//       }

Name: Syntax Error.
Message: 
StackTrace: 

In [19]:
// evaluations.sorted.reverse.foreach(println)

Name: Syntax Error.
Message: 
StackTrace: 

In [20]:
// evaluations.sorted.foreach(println)

Name: Syntax Error.
Message: 
StackTrace: 

In [21]:
// val rank =evaluations.sorted.head._2._1
// val regParam=evaluations.sorted.head._2._2 
// val alpha = evaluations.sorted.head._2._3

Name: Syntax Error.
Message: 
StackTrace: 

In [22]:
val rank = 30
val regParam = 1.0E-4
val alpha = 40.0
val model = new ALS().
          setSeed(Random.nextLong()).
          setImplicitPrefs(true).
          setRank(rank).setRegParam(regParam).
          setAlpha(alpha).setMaxIter(20).
          setUserCol("user").setItemCol("artist").
          setRatingCol("count").
          setColdStartStrategy("drop").
          fit(trainData)
val predictions = model.transform(cvData)
val evaluator = new RegressionEvaluator().setMetricName("rmse").setLabelCol("count").setPredictionCol("prediction")
val rmse = evaluator.evaluate(predictions)

rank = 30
regParam = 1.0E-4
alpha = 40.0
model = als_bb123a861b44
predictions = [user: int, artist: int ... 2 more fields]
evaluator = regEval_55c0d1b6b76d
rmse = 294.1692232344606


294.1692232344606

In [23]:
predictions.show(5, false)

+-------+------+-----+----------+
|user   |artist|count|prediction|
+-------+------+-----+----------+
|1042671|463   |41   |1.0113472 |
|1064119|463   |2    |0.9421004 |
|1070243|463   |1    |0.9754186 |
|2017486|463   |3    |0.9949184 |
|2033127|463   |39   |1.0008838 |
+-------+------+-----+----------+
only showing top 5 rows



In [24]:
cvData.show(5,false)

+----+--------+-----+
|user|artist  |count|
+----+--------+-----+
|116 |785     |3    |
|120 |463     |1    |
|120 |1012523 |1    |
|120 |10647021|1    |
|1014|979     |3    |
+----+--------+-----+
only showing top 5 rows



In [32]:
// Generate top 10 movie recommendations for a specified set of users
val users = cvData.select(model.getUserCol).distinct().limit(10)

users = [user: int]


[user: int]

In [33]:
val userSubsetRecs = model.recommendForUserSubset(users, 10)

userSubsetRecs = [user: int, recommendations: array<struct<artist:int,rating:float>>]


[user: int, recommendations: array<struct<artist:int,rating:float>>]

In [34]:
// Generate top 10 user recommendations for a specified set of artists
val movies = cvData.select(model.getItemCol).distinct().limit(10)

movies = [artist: int]


[artist: int]

In [35]:
val movieSubSetRecs = model.recommendForItemSubset(movies, 10)

movieSubSetRecs = [artist: int, recommendations: array<struct<user:int,rating:float>>]


[artist: int, recommendations: array<struct<user:int,rating:float>>]

In [36]:
userSubsetRecs.show()
movieSubSetRecs.show()

+-------+--------------------+
|   user|     recommendations|
+-------+--------------------+
|1000092|[[1003214, 1.3375...|
|1002892|[[506, 1.2277234]...|
|1002401|[[1034635, 0.7877...|
|1000614|[[4629, 1.1762543...|
|1001824|[[4629, 1.3214068...|
|1000693|[[1000113, 0.9633...|
|1000349|[[1002782, 1.1347...|
|1000565|[[615, 1.1303912]...|
|1001375|[[1250092, 1.2189...|
|1002195|[[1037206, 1.1342...|
+-------+--------------------+

+-------+--------------------+
| artist|     recommendations|
+-------+--------------------+
|1006924|[[2037213, 1.4493...|
|1322464|[[2051047, 1.3214...|
|2152724|[[2024005, 0.5749...|
|    137|[[2131285, 1.5404...|
|1023157|[[1044581, 1.2914...|
|   2923|[[1059334, 1.2451...|
|1128583|[[2166359, 1.2187...|
|   2259|[[1044648, 1.7408...|
|1275899|[[2037213, 1.4853...|
|1327749|[[1043751, 0.7366...|
+-------+--------------------+



In [39]:
userSubsetRecs.selectExpr("user","explode(recommendations)").show(20, false)

+-------+---------------------+
|user   |col                  |
+-------+---------------------+
|1000144|[1037447, 1.3648036] |
|1000144|[1011231, 1.3306216] |
|1000144|[1008629, 1.256475]  |
|1000144|[1301399, 1.2554127] |
|1000144|[1010695, 1.2514899] |
|1000144|[1051076, 1.2150977] |
|1000144|[1012005, 1.210488]  |
|1000144|[1022927, 1.2059901] |
|1000144|[2160922, 1.1977018] |
|1000144|[914, 1.1943628]     |
|1001410|[1034635, 0.43859312]|
|1001410|[1000113, 0.43616775]|
|1001410|[979, 0.4360511]     |
|1001410|[1177, 0.43496805]   |
|1001410|[1231740, 0.43461528]|
|1001410|[352, 0.43445012]    |
|1001410|[189, 0.43409985]    |
|1001410|[1205, 0.43408656]   |
|1001410|[1270639, 0.43386883]|
|1001410|[1307, 0.43364018]   |
+-------+---------------------+
only showing top 20 rows



In [40]:
movieSubSetRecs.selectExpr("artist","explode(recommendations)").show(20, false)

+--------+----------------------+
|artist  |col                   |
+--------+----------------------+
|10628801|[2051151, 0.124753475]|
|10628801|[2062682, 0.1136824]  |
|10628801|[1070844, 0.11345446] |
|10628801|[1004646, 0.10658086] |
|10628801|[1052918, 0.098159686]|
|10628801|[2078841, 0.09698816] |
|10628801|[2053493, 0.09281278] |
|10628801|[2009373, 0.092252836]|
|10628801|[2049131, 0.08965781] |
|10628801|[2025300, 0.08833962] |
|1017864 |[1061434, 0.89051497] |
|1017864 |[1026084, 0.79273033] |
|1017864 |[2016026, 0.79050434] |
|1017864 |[1062331, 0.77775455] |
|1017864 |[1001440, 0.7684743]  |
|1017864 |[1050213, 0.7631943]  |
|1017864 |[1021827, 0.74281824] |
|1017864 |[2055171, 0.728516]   |
|1017864 |[2135148, 0.7182686]  |
|1017864 |[1006261, 0.69207144] |
+--------+----------------------+
only showing top 20 rows

